In [ ]:
# CHM N-BODY (grid 4D) in PyTorch (CUDA) + 2x2 render (XY/XZ/YZ + XT)

import os, shutil, subprocess, math
import numpy as np
import matplotlib.pyplot as plt
import torch

# =============================================================================
# What this script does (plain-English)
# -----------------------------------------------------------------------------

# This project implements an explicit abstract hyperedge: a k-ary relational event integrated by # a kernel reducible to pairwise operations (hardware constraint).
# Time is not a primitive; only hyperedge activation is real.

# We simulate N "particles" living in a 4D unit box: (x, y, z, t) in [0, 1).
# - The particles are indexed on a 4D grid (gt, gz, gy, gx), so N = H*W*D*T.
# - Each particle interacts only with a small local neighborhood on that *index grid*.
# - The interaction is a k-ary "attention kernel":
#     * compute a weighted centroid of neighbor positions
#     * compute a weighted average of neighbor velocities
#     * add a short-range repulsion to avoid collapse
# - Update velocity and position each step, with damping and periodic boundaries.
#
# Visualization:
# - We do not try to build a full 4D histogram (too big).
# - Instead we render 2D density projections from the 4D cloud:
#     XY (ignoring Z,T), XZ (ignoring Y,T), YZ (ignoring X,T), XT (ignoring Y,Z)
# - Optional: rotate only XYZ for the view; keep the 4th coordinate (t) unchanged.
# =============================================================================


# =============================================================================
# Main config (4D)
# =============================================================================
H, W, D, T = 32, 32, 32, 16   # grid dimensions for indexing particles (gt,gz,gy,gx)
N = H * W * D * T

STEPS = 120                   # simulation steps
DT = 0.06                     # integration step size
SEED = 9                      # RNG seed (torch Generator)


# =============================================================================
# Neighborhood on the *index grid* (4D)
# -----------------------------------------------------------------------------
# RADIUS defines the local offsets we consider in (dt, dz, dy, dx).
# We keep at most K nearest offsets (smallest Euclidean distance on the index grid).
# =============================================================================
K = 32
RADIUS = 2


# =============================================================================
# Kernel weights ("Riemann-like", real-valued)
# -----------------------------------------------------------------------------
# Weight as a function of offset distance r (on the index grid):
#   w(r) ~ (r + 1)^(-1/2) * exp(-r^2 / (2*sigma^2))
#
# This is just a stable real weighting: strong nearby influence, soft long tail.
# =============================================================================
SIGMA = 1.6
EPS = 1e-6


# =============================================================================
# Dynamics (forces / couplings)
# =============================================================================
ATTRACTION = 1.20   # pull toward neighbor centroid
ALIGN      = 0.22   # match neighbor average velocity
REPULSION  = 0.020  # short-range repulsion (prevents clumping)
DAMP       = 0.06   # velocity damping (energy leak)
VCLIP      = 2.0    # clamp velocity magnitude per component (cheap stability)


# =============================================================================
# Render / output
# =============================================================================
SAVE_FRAMES = True
FRAMES_DIR  = "frames_chm_nbody_4d_torch"
MAKE_MP4    = True
MP4_PATH    = "chm_nbody_4d_torch.mp4"
FPS         = 30

BINS = 200   # histogram bins for each 2D panel (200x200)


# =============================================================================
# View rotation (render-only)
# -----------------------------------------------------------------------------
# Rotate XYZ around the cube center (0.5,0.5,0.5). Keep the 4th coord "t" intact.
# This is purely cosmetic so you can see 3D structure move in the projections.
# =============================================================================
ROTATE_VIEW = True
ROT_SPEED   = 0.035
ROT_AXES    = (0.9, 0.6, 0.2)


# =============================================================================
# Device
# =============================================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device, "| N:", N)


# =============================================================================
# Helpers
# =============================================================================
def build_neighbor_offsets_4d(radius: int):
    """
    Enumerate all 4D integer offsets (dt,dz,dy,dx) within a ball of given radius,
    excluding (0,0,0,0). Also compute the Euclidean distance of each offset.

    Returns a list of tuples: (dt, dz, dy, dx, dist).
    """
    offs = []
    for dt_ in range(-radius, radius + 1):
        for dz_ in range(-radius, radius + 1):
            for dy_ in range(-radius, radius + 1):
                for dx_ in range(-radius, radius + 1):
                    if dx_ == 0 and dy_ == 0 and dz_ == 0 and dt_ == 0:
                        continue

                    d = (dx_*dx_ + dy_*dy_ + dz_*dz_ + dt_*dt_) ** 0.5
                    if d <= radius + 1e-9:
                        offs.append((dt_, dz_, dy_, dx_, d))

    # Sort by distance so we can take the K closest offsets
    offs.sort(key=lambda t: t[4])
    return offs


def riemann_like_weight_torch(r: torch.Tensor, sigma: float):
    """
    Real weight function:
      w(r) = (r + 1)^(-1/2) * exp(-r^2 / (2*sigma^2))

    r is a tensor of distances (>= 0).
    """
    return (r + 1.0).pow(-0.5) * torch.exp(-(r * r) / (2.0 * sigma * sigma))


@torch.no_grad()
def rotate_xyz_keep_t(p_xyzt: torch.Tensor, step: int, speed: float, axes=(1.0, 1.0, 1.0)):
    """
    Rotate ONLY xyz around the center of the unit cube; keep t unchanged.
    Used only for rendering, not for the simulation itself.
    """
    xyz = p_xyzt[:, :3]
    tt  = p_xyzt[:, 3:4]

    a = float(step) * float(speed)
    ax, ay, az = axes
    ax = a * float(ax)
    ay = a * float(ay)
    az = a * float(az)

    cx, sx = math.cos(ax), math.sin(ax)
    cy, sy = math.cos(ay), math.sin(ay)
    cz, sz = math.cos(az), math.sin(az)

    # Basic XYZ Euler-ish rotation (Rx then Ry then Rz)
    Rx = torch.tensor([[1.0, 0.0, 0.0],
                       [0.0,  cx, -sx],
                       [0.0,  sx,  cx]], device=p_xyzt.device, dtype=p_xyzt.dtype)
    Ry = torch.tensor([[ cy, 0.0,  sy],
                       [0.0, 1.0, 0.0],
                       [-sy, 0.0,  cy]], device=p_xyzt.device, dtype=p_xyzt.dtype)
    Rz = torch.tensor([[ cz, -sz, 0.0],
                       [ sz,  cz, 0.0],
                       [0.0, 0.0, 1.0]], device=p_xyzt.device, dtype=p_xyzt.dtype)

    R = (Rz @ Ry @ Rx)

    # Center -> rotate -> wrap back into [0,1)
    q = xyz - 0.5
    q = q @ R.T
    q = q + 0.5
    q = torch.remainder(q, 1.0)

    return torch.cat([q, tt], dim=1)


@torch.no_grad()
def hist2d_counts(pos_xyzt: torch.Tensor, bins: int, a: int, b: int):
    """
    Fast GPU 2D histogram for the selected dims a,b of pos_xyzt (each in [0,1)).

    Returns a (bins, bins) tensor of counts.
    Note: torch.bincount returns a flat vector; we reshape back to 2D.
    """
    B = int(bins)

    # Map [0,1) -> integer bin index [0, B-1]
    uv = (pos_xyzt[:, [a, b]] * B).to(torch.int64)
    uv = torch.clamp(uv, 0, B - 1)

    u = uv[:, 0]
    v = uv[:, 1]

    # Flatten 2D bins (u,v) -> idx
    idx = u + B * v

    # bincount gives counts per bin index
    counts = torch.bincount(idx, minlength=B * B).reshape(B, B)
    return counts


@torch.no_grad()
def render_density_2x2(pos_xyzt: torch.Tensor, out_path: str, bins: int, step: int):
    """
    Render a 2x2 grid of log-density projections from the 4D cloud.

    Panels:
      - XY: dims (x,y)
      - XZ: dims (x,z)
      - YZ: dims (y,z)
      - XT: dims (x,t)
    """
    if ROTATE_VIEW:
        p_vis = rotate_xyz_keep_t(pos_xyzt, step=step, speed=ROT_SPEED, axes=ROT_AXES)
    else:
        p_vis = pos_xyzt

    # dim indices: 0=x, 1=y, 2=z, 3=t
    xy = torch.log1p(hist2d_counts(p_vis, bins, 0, 1)).T
    xz = torch.log1p(hist2d_counts(p_vis, bins, 0, 2)).T
    yz = torch.log1p(hist2d_counts(p_vis, bins, 1, 2)).T
    xt = torch.log1p(hist2d_counts(p_vis, bins, 0, 3)).T

    # Move to CPU for matplotlib
    xy_cpu = xy.float().cpu().numpy()
    xz_cpu = xz.float().cpu().numpy()
    yz_cpu = yz.float().cpu().numpy()
    xt_cpu = xt.float().cpu().numpy()

    plt.figure(figsize=(8, 8), dpi=140)

    ax1 = plt.subplot(2, 2, 1)
    ax1.imshow(xy_cpu, origin="lower", extent=[0, 1, 0, 1], interpolation="nearest")
    ax1.set_title("XY (ignore Z,T)", fontsize=10)
    ax1.axis("off")

    ax2 = plt.subplot(2, 2, 2)
    ax2.imshow(xz_cpu, origin="lower", extent=[0, 1, 0, 1], interpolation="nearest")
    ax2.set_title("XZ (ignore Y,T)", fontsize=10)
    ax2.axis("off")

    ax3 = plt.subplot(2, 2, 3)
    ax3.imshow(yz_cpu, origin="lower", extent=[0, 1, 0, 1], interpolation="nearest")
    ax3.set_title("YZ (ignore X,T)", fontsize=10)
    ax3.axis("off")

    ax4 = plt.subplot(2, 2, 4)
    ax4.imshow(xt_cpu, origin="lower", extent=[0, 1, 0, 1], interpolation="nearest")
    ax4.set_title("XT (ignore Y,Z)", fontsize=10)
    ax4.axis("off")

    plt.tight_layout(pad=0.2)
    plt.savefig(out_path, bbox_inches="tight", pad_inches=0.02)
    plt.close()


# =============================================================================
# Build neighbor offsets + weights (k-ary attention stencil)
# =============================================================================
offs = build_neighbor_offsets_4d(RADIUS)
if len(offs) == 0:
    raise RuntimeError("RADIUS too small: no neighbor offsets found.")

# Keep only the first K closest offsets
offs = offs[:K]

dt = torch.tensor([o[0] for o in offs], dtype=torch.int64, device=device)
dz = torch.tensor([o[1] for o in offs], dtype=torch.int64, device=device)
dy = torch.tensor([o[2] for o in offs], dtype=torch.int64, device=device)
dx = torch.tensor([o[3] for o in offs], dtype=torch.int64, device=device)
dist = torch.tensor([o[4] for o in offs], dtype=torch.float32, device=device)

# Convert distances -> normalized weights
w = riemann_like_weight_torch(dist, SIGMA)
w = w / (w.sum() + EPS)   # shape (K,)


# =============================================================================
# Map linear index i -> 4D grid coordinates (gt,gz,gy,gx)
# -----------------------------------------------------------------------------
# i = (((gt * D + gz) * H + gy) * W + gx)
#
# These are *indexing coordinates*, not physical positions.
# The actual particle position is pos[i] in continuous (x,y,z,t) space.
# =============================================================================
gx = torch.arange(W, device=device, dtype=torch.int64).repeat(T * D * H)
gy = torch.arange(H, device=device, dtype=torch.int64).repeat_interleave(W).repeat(T * D)
gz = torch.arange(D, device=device, dtype=torch.int64).repeat_interleave(H * W).repeat(T)
gt = torch.arange(T, device=device, dtype=torch.int64).repeat_interleave(D * H * W)

# Neighbor indices for each particle with periodic wrap on the index grid
nt = (gt[:, None] + dt[None, :]) % T
nz = (gz[:, None] + dz[None, :]) % D
ny = (gy[:, None] + dy[None, :]) % H
nx = (gx[:, None] + dx[None, :]) % W

neigh_idx = (((nt * D + nz) * H + ny) * W + nx)  # shape (N, K)


# =============================================================================
# Initialize positions and velocities (pure random)
# -----------------------------------------------------------------------------
# pos lives in continuous 4D unit box: [0,1) for each dimension (x,y,z,t).
# vel starts as small random values.
# =============================================================================
g = torch.Generator(device=device)
g.manual_seed(SEED)

pos = torch.rand((N, 4), generator=g, device=device, dtype=torch.float32)
vel = (torch.rand((N, 4), generator=g, device=device, dtype=torch.float32) * 2.0 - 1.0) * 0.25


# =============================================================================
# Prepare output directory
# =============================================================================
if SAVE_FRAMES:
    if os.path.exists(FRAMES_DIR):
        shutil.rmtree(FRAMES_DIR)
    os.makedirs(FRAMES_DIR, exist_ok=True)


# =============================================================================
# Main loop
# =============================================================================
for step in range(STEPS):
    # Gather neighbor positions/velocities: (N, K, 4)
    p_n = pos[neigh_idx]
    v_n = vel[neigh_idx]

    # Weighted neighbor centroid and velocity mean
    ww = w.view(1, K, 1)               # broadcast shape (1, K, 1)
    centroid = (ww * p_n).sum(dim=1)   # (N, 4)
    vbar     = (ww * v_n).sum(dim=1)   # (N, 4)

    # Short-range repulsion in a periodic 4D box:
    # Compute minimum-image displacement dp in [-0.5, 0.5) per coordinate.
    p_i = pos.unsqueeze(1)             # (N, 1, 4)
    dp = p_i - p_n                     # (N, K, 4)
    dp = torch.remainder(dp + 0.5, 1.0) - 0.5

    # Inverse-square-ish repulsion (softened by EPS)
    r2 = (dp * dp).sum(dim=2) + EPS    # (N, K)
    rep = (dp / r2.unsqueeze(2)).sum(dim=1)  # (N, 4)

    # Combine terms into acceleration
    acc = (
        ATTRACTION * (centroid - pos) +
        ALIGN      * (vbar - vel) +
        REPULSION  * rep
    )

    # Semi-implicit Euler-ish update with damping and clamp for stability
    vel = vel + DT * acc
    vel = vel * (1.0 - DAMP)
    vel = torch.clamp(vel, -VCLIP, VCLIP)

    pos = pos + DT * vel
    pos = torch.remainder(pos, 1.0)  # periodic boundary in continuous space

    # Render frame
    if SAVE_FRAMES:
        out = os.path.join(FRAMES_DIR, f"frame_{step:05d}.png")
        render_density_2x2(pos, out, bins=BINS, step=step)

    if (step + 1) % 20 == 0:
        print(f"step {step+1}/{STEPS}")


print("Done.")

device: cpu | N: 524288


In [ ]:
!ffmpeg -y \
  -framerate 30 \
  -i "frames_chm_nbody_4d_torch/frame_%05d.png" \
  -c:v libx264 \
  -preset medium \
  -crf 18 \
  "A 4-Dimensional Hyperedge.mp4"